### This Notebook has been written as part of the Group Project for ELEC4546 Investing and Trading for Engineering Students

In [1]:
#!pip install stocktrends
#!pip install numpy
#!pip install pandas
#!pip install matplotlib
#!pip install statsmodels
import numpy as np
from stocktrends import Renko
import statsmodels.api as sm
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import time

### The following section implements functions for various technical indicators that are used in the industry.
### These indicators will then be used to create strategies 

1. Moving Average Convergence Divergence (MACD)

In [2]:
def MACD(DF,fast_MA,slow_MA,macd_signal_period):
    """function to calculate MACD
       Returns the trend indicated: either bullish or bearish
       typical values: fast_MA = 12; slow_MA =26, macd_signal_period =9
       However, user can enter their own values
       The exponential moving average has been used here to give more weightage to recent prices"""
    df = DF.copy()
    df["MA_Fast"]=df["Adj Close"].ewm(span=fast_MA,min_periods=fast_MA).mean() # fast moving average
    df["MA_Slow"]=df["Adj Close"].ewm(span=slow_MA,min_periods=slow_MA).mean() # slow moving average
    df["MACD"]=df["MA_Fast"]-df["MA_Slow"] # MACD line
    df["Signal"]=df["MACD"].ewm(span=macd_signal_period,min_periods=macd_signal_period).mean() # Signal Line
    #df.dropna(inplace=True)
    df["MACD_Shifted"]=df["MACD"].shift(1)
    df["Signal_Shifted"]=df["Signal"].shift(1)
    #df.dropna(inplace=True)
    
    df.loc[(df["MACD"]>df["Signal"]) & (df["MACD_Shifted"]<df["Signal_Shifted"]),'Trend']='Bullish'
    df.loc[(df["MACD"]<df["Signal"]) & (df["MACD_Shifted"]>df["Signal_Shifted"]),'Trend']='Bearish'
    return df["Trend"]

2. Average True Range (ATR)

In [3]:
def ATR(DF,n):
    """function to calculate True Range and Average True Range
       n is the period"""
    df = DF.copy()
    df['High-Low']=abs(df['High']-df['Low']) # Difference between high and low price
    df['High-PreviousClose']=abs(df['High']-df['Adj Close'].shift(1)) # Difference between high and prev_close
    df['Low-PreviousClose']=abs(df['Low']-df['Adj Close'].shift(1)) # Difference between low and prev_close
    df['TR']=df[['High-Low','High-PreviousClose','Low-PreviousClose']].max(axis=1,skipna=False) # Take the maximum of the above 3
    df['ATR'] = df['TR'].rolling(n).mean() # Calculate a rolling moving average based on the number of periods chosen by user
    #df['ATR'] = df['TR'].ewm(span=n,adjust=False,min_periods=n).mean() Can also use an exponential moving average
    df2 = df.drop(['High-Low','High-PreviousClose','Low-PreviousClose'],axis=1)
    return df2

3. Bollinger Bands

In [4]:
def BollBnd(DF,n):
    """function to calculate Bollinger Band
       n is the period"""
    df = DF.copy()
    df["MA"] = df['Adj Close'].rolling(n).mean()
    df["BB_up"] = df["MA"] + 2*df['Adj Close'].rolling(n).std(ddof=0) # We take a population standard deviation
    df["BB_down"] = df["MA"] - 2*df['Adj Close'].rolling(n).std(ddof=0) # We take a population standard deviation
    df["BB_width"] = df["BB_up"] - df["BB_down"] # This gices the width of the Bollinger Band
    df.dropna(inplace=True)
    return df["BB_up"],df["BB_down"],df["BB_width"]

4. Relative Strength Index (RSI)

In [5]:
def RSI(DF,n):
    """function to calculate RSI
       n is the period"""
    df = DF.copy()
    df['delta']=df['Adj Close'] - df['Adj Close'].shift(1)
    df['gain']=np.where(df['delta']>=0,df['delta'],0) # Gain if difference between close and previous close is positive
    df['loss']=np.where(df['delta']<0,abs(df['delta']),0) # Loss if difference between close and previous close is negative
    avg_gain = []
    avg_loss = []
    gain = df['gain'].tolist()
    loss = df['loss'].tolist()
    for i in range(len(df)):
        if i < n:
            avg_gain.append(np.NaN)
            avg_loss.append(np.NaN)
        elif i == n:
            avg_gain.append(df['gain'].rolling(n).mean().tolist()[n])
            avg_loss.append(df['loss'].rolling(n).mean().tolist()[n])
        elif i > n:
            avg_gain.append(((n-1)*avg_gain[i-1] + gain[i])/n)
            avg_loss.append(((n-1)*avg_loss[i-1] + loss[i])/n)
    df['avg_gain']=np.array(avg_gain)
    df['avg_loss']=np.array(avg_loss)
    df['RS'] = df['avg_gain']/df['avg_loss']
    df['RSI'] = 100 - (100/(1+df['RS']))
    return df['RSI']

5. Average Directional Index (ADX)

In [6]:
def ADX(DF,n):
    """function to calculate ADX
       n is the period
    """
     
    df2 = DF.copy()
    df2['TR'] = ATR(df2,n)['TR'] #the period parameter of ATR function does not matter because period does not influence TR calculation
    df2['DMplus']=np.where((df2['High']-df2['High'].shift(1))>(df2['Low'].shift(1)-df2['Low']),df2['High']-df2['High'].shift(1),0)
    df2['DMplus']=np.where(df2['DMplus']<0,0,df2['DMplus'])
    df2['DMminus']=np.where((df2['Low'].shift(1)-df2['Low'])>(df2['High']-df2['High'].shift(1)),df2['Low'].shift(1)-df2['Low'],0)
    df2['DMminus']=np.where(df2['DMminus']<0,0,df2['DMminus'])
    TRn = []
    DMplusN = []
    DMminusN = []
    TR = df2['TR'].tolist()
    DMplus = df2['DMplus'].tolist()
    DMminus = df2['DMminus'].tolist()
    for i in range(len(df2)):
        if i < n:
            TRn.append(np.NaN)
            DMplusN.append(np.NaN)
            DMminusN.append(np.NaN)
        elif i == n:
            TRn.append(df2['TR'].rolling(n).sum().tolist()[n])
            DMplusN.append(df2['DMplus'].rolling(n).sum().tolist()[n])
            DMminusN.append(df2['DMminus'].rolling(n).sum().tolist()[n])
        elif i > n:
            TRn.append(TRn[i-1] - (TRn[i-1]/n) + TR[i])
            DMplusN.append(DMplusN[i-1] - (DMplusN[i-1]/n) + DMplus[i])
            DMminusN.append(DMminusN[i-1] - (DMminusN[i-1]/n) + DMminus[i])
    df2['TRn'] = np.array(TRn)
    df2['DMplusN'] = np.array(DMplusN)
    df2['DMminusN'] = np.array(DMminusN)
    df2['DIplusN']=100*(df2['DMplusN']/df2['TRn'])
    df2['DIminusN']=100*(df2['DMminusN']/df2['TRn'])
    df2['DIdiff']=abs(df2['DIplusN']-df2['DIminusN'])
    df2['DIsum']=df2['DIplusN']+df2['DIminusN']
    df2['DX']=100*(df2['DIdiff']/df2['DIsum'])
    ADX = []
    DX = df2['DX'].tolist()
    for j in range(len(df2)):
        if j < 2*n-1:
            ADX.append(np.NaN)
        elif j == 2*n-1:
            ADX.append(df2['DX'][j-n+1:j+1].mean())
        elif j > 2*n-1:
            ADX.append(((n-1)*ADX[j-1] + DX[j])/n)
    df2['ADX']=np.array(ADX)
    return df2['ADX']

6. On Balance Volume (OBV)

In [7]:
def OBV(DF):
    """function to calculate On Balance Volume"""
    df = DF.copy()
    df['daily_ret'] = df['Adj Close'].pct_change()
    df['direction'] = np.where(df['daily_ret']>=0,1,-1)
    df['direction'][0] = 0
    df['vol_adj'] = df['Volume'] * df['direction']
    df['obv'] = df['vol_adj'].cumsum()
    return df['obv']

7. Calculating Slope of a Line with n points 

In [8]:
def slope(points,n):
    "function to calculate the slope of regression line for n consecutive points on a plot"
    slopes = [i*0 for i in range(n-1)]
    for i in range(n,len(points)+1):
        y = points[i-n:i]
        x = np.array(range(n))
        # Scaling to normalize the points 
        y_scaled = (y - y.min())/(y.max() - y.min())
        x_scaled = (x - x.min())/(x.max() - x.min())
        x_scaled = sm.add_constant(x_scaled)
        model = sm.OLS(y_scaled,x_scaled)
        results = model.fit() # Gives the best fit line
        slopes.append(results.params[-1]) # This gives the slope in radians
    slope_angle = (np.rad2deg(np.arctan(np.array(slopes)))) #Convert slope to degrees and then use tan inverse to get numerical value
    return np.array(slope_angle)

8. Renko Charts

In [9]:
def renko_DF(DF):
    "function to convert ohlc data into renko bricks"
    df = DF.copy()
    df.reset_index(inplace=True)
    df = df.iloc[:,[0,1,2,3,4,5]]
    df.columns = ["date","open","close","high","low","volume"]
    df2 = Renko(df)
    df2.brick_size = round(ATR(DF,120)["ATR"][-1],4)
    renko_df = df2.get_bricks()
    renko_df["bar_num"] = np.where(renko_df["uptrend"]==True,1,np.where(renko_df["uptrend"]==False,-1,0))
    for i in range(1,len(renko_df["bar_num"])):
        if renko_df["bar_num"][i]>0 and renko_df["bar_num"][i-1]>0:
            renko_df["bar_num"][i]+=renko_df["bar_num"][i-1]
        elif renko_df["bar_num"][i]<0 and renko_df["bar_num"][i-1]<0:
            renko_df["bar_num"][i]+=renko_df["bar_num"][i-1]
    renko_df.drop_duplicates(subset="date",keep="last",inplace=True)
    return renko_df

9. Stochastic Indicator

In [10]:
def stochastic(df,a,b,c):
    "function to calculate stochastic"
    df['k']=((df['Adj Close'] - df['Low'].rolling(a).min())/(df['High'].rolling(a).max()-df['Low'].rolling(a).min()))*100
    df['K']=df['k'].rolling(b).mean() 
    df['D']=df['K'].rolling(c).mean()
    return df

10. SMA Indicator

In [11]:
def SMA(df,a,b):
    "function to calculate stochastic"
    df['sma_fast']=df['Adj Close'].rolling(a).mean() 
    df['sma_slow']=df['Adj Close'].rolling(b).mean() 
    return df

### Next, some functions for performance metrics are implemented

1. Cumulative Annual Growth Rate

In [12]:
def CAGR(DF,x):
    """function to calculate the Cumulative Annual Growth Rate of a trading strategy
       x is the number of times the strategy runs in a day"""
    df = DF.copy()
    df["cum_return"] = (1 + df["Return"]).cumprod()
    n = len(df)/(252*x) 
    CAGR = (df["cum_return"].tolist()[-1])**(1/n) - 1
    return CAGR

2. Volatility (Standard Deviation)

In [13]:
def volatility(DF,x):
    """function to calculate annualized volatility of a trading strategy
       x is the number of times the strategy will run in a day"""
    df = DF.copy()
    vol = df["Return"].std() * np.sqrt(252*x)
    return vol


3. Sharpe Ratio

In [14]:
def sharpe(DF,rf,x):
    """function to calculate sharpe ratio
      rf is the risk free rate
      x is the number of times the strategy will run in a day"""
    df = DF.copy()
    sr = (CAGR(df,x) - rf)/volatility(df,x)
    return sr

4. Maximum Drawdown

In [15]:
def max_dd(DF):
    "function to calculate max drawdown"
    df = DF.copy()
    df["cum_return"] = (1 + df["Return"]).cumprod()
    df["cum_roll_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_roll_max"] - df["cum_return"]
    df["drawdown_pct"] = df["drawdown"]/df["cum_roll_max"]
    max_dd = df["drawdown_pct"].max()
    return max_dd

### Establish a connection with the OANDA Forex Trading Platform using their API

In [16]:
!pip install oandapyV20
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades

# Enter your OANDA Login Details here

In [17]:
#initiating API connection and defining trade parameters
token = "0fcb2b978872704ea19d39f4505bf1d4-5b3a21e7af287feb8d9efc2a559c4448"
client = oandapyV20.API(token,environment="practice")
account_id = "101-001-16805203-001"

### Function to create a simple market order

In [18]:
def market_order(instrument,units,account_id):
    """units can be positive or negative"""
    
    data = {
            "order": {
            "price": "1.19",
            "timeInForce": "GTC",
            "instrument": str(instrument),
            "units": str(units),
            "type": "LIMIT",
            "positionFill": "DEFAULT"
                    }
            }
    r = orders.OrderCreate(accountID=account_id, data=data)
    client.request(r) 

### Function to pull Open, Low, High, Adj Close, and Volume Data from OANDA

In [19]:
def candles(instrument):
    params = {"count": "1000","granularity": "M5"} #granularity can be in seconds S5 - S30, minutes M1 - M30, hours H1 - H12, days D, weeks W or months M
    candles = instruments.InstrumentsCandles(instrument=instrument,params=params)
    client.request(candles)
    ohlc_dict = candles.response["candles"]
    ohlc = pd.DataFrame(ohlc_dict)
    ohlc_df = ohlc.mid.dropna().apply(pd.Series)
    ohlc_df["volume"]=ohlc["volume"]
    ohlc_df.index = ohlc["time"]
    ohlc_df = ohlc_df.apply(pd.to_numeric)
    ohlc_df.columns=["Open","High","Low","Adj Close","Volume"]
    return ohlc_df

### Trading Account Details

In [20]:
def account_details():
    """ 
    Provides all details of the account such as:
    1. Basic details: settings, account_id, created_Time, currency, balance, number of openPositions, OpenTrades and pendingOrders
    2. Details of orders
    3. Details of positions
    4. Details of trades"""
    r = accounts.AccountDetails(accountID=account_id)
    client.request(r)
    return r.response

### Trading Account Summary

In [21]:
def account_summary():
    """
    Provides Basic details: settings, account_id, created_Time, currency, balance, number of openPositions, OpenTrades and 
    pendingOrders
    """
    r = accounts.AccountSummary(accountID=account_id)
    client.request(r)
    return r.response

### Check all open trades

In [22]:
def trade():
    r = trades.OpenTrades(accountID=account_id)
    return client.request(r)

### Strategy 1: SMA Crossover

In [27]:
upward_sma_dir = {}
dnward_sma_dir = {}
pairs = ['EUR_USD','GBP_USD','AUD_USD','USD_JPY','EUR_GBP']
for i in pairs:
    upward_sma_dir[i] = False
    dnward_sma_dir[i] = False

In [45]:
def trade_signalSMA(df,curr):
    "function to generate signal"
    global upward_sma_dir, dnward_sma_dir
    signal = ""
    if df['sma_fast'][-1] < df['sma_slow'][-1] and df['sma_fast'][-2] < df['sma_slow'][-2]:
        upward_sma_dir[curr] = True
        dnward_sma_dir[curr] = False
    if df['sma_fast'][-1] < df['sma_slow'][-1] and df['sma_fast'][-2] > df['sma_slow'][-2]:
        upward_sma_dir[curr] = False
        dnward_sma_dir[curr] = True  
    if upward_sma_dir[curr] == True and min(df['K'][-1],df['D'][-1]) > 25 and max(df['K'][-2],df['D'][-2]) < 25:
        signal = "Buy"
    if dnward_sma_dir[curr] == True and min(df['K'][-1],df['D'][-1]) > 75 and max(df['K'][-2],df['D'][-2]) < 75:
        signal = "Sell"
    return signal

### Strategy 2: MACD-ADX

In [42]:
def trade_signalMACD_ADX(df):
    signal=""
    if df['adx'][-1]>25 and df['macd'][-1]=='Bullish':
        signal="Buy"
    if df['adx'][-1]>25 and df['macd'][-1]=='Bearish':
        signal="Sell"
    return signal      

### Running the Strategies

In [43]:
def main(account_id):
    pairs = ['EUR_USD','GBP_USD','AUD_USD','USD_JPY','EUR_GBP'] # Currency pairs selected
    pos_size=10000
    try:
        for currency in pairs:
            print("analyzing ",currency)
            data = candles(currency)
            ohlc_df_SMA=data.copy()
            ohlc_df_SMA = stochastic(ohlc_df_SMA,14,3,3)
            ohlc_df_SMA = SMA(ohlc_df_SMA,100,200)
            SMAsignal = trade_signalSMA(ohlc_df_SMA,currency)
            
            ohlc_df_MACD_ADX=data.copy()
            ohlc_df_MACD_ADX["macd"]=MACD(ohlc_df_MACD_ADX,25,99,45)
            ohlc_df_MACD_ADX["adx"]=ADX(ohlc_df_MACD_ADX,20)

           # print(ohlc_df_MACD_ADX)
            MACD_ADX_Signal=trade_signalMACD_ADX(ohlc_df_MACD_ADX)
           # print('MACD_ADX_Signal:',MACD_ADX_Signal)
            
            if SMAsignal == "Buy" or MACD_ADX_Signal=="Buy":
                market_order(currency,pos_size,account_id)
                print("New long position initiated for ", currency)
            elif SMAsignal == "Sell" or MACD_ADX_Signal=="Sell":
                market_order(currency,-1*pos_size, account_id)
                print("New short position initiated for ", currency)
    except:
        print("error encountered....skipping this iteration")

# Fix this function to run the code every 5 minutes before deploying

In [44]:
# Continuous execution
#main(account_id)
starttime=time.time()
timeout = time.time() + 60*1  # 60 seconds times 60 meaning the script will run for 1 hr
while time.time() <= timeout:
    try:
        print("passthrough at ",time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
        main(account_id)
        time.sleep(60 - ((time.time() - starttime) % 60.0)) # 5 minute interval between each new execution
    except KeyboardInterrupt:
        print('\n\nKeyboard exception received. Exiting.')
        exit()

passthrough at  2020-11-19 20:20:21
analyzing  EUR_USD
New long position initiated for  EUR_USD
analyzing  GBP_USD
New long position initiated for  GBP_USD
analyzing  AUD_USD
New long position initiated for  AUD_USD
analyzing  USD_JPY
New long position initiated for  USD_JPY
analyzing  EUR_GBP
New long position initiated for  EUR_GBP


### Work to be done for final report:
1. Pull trade history from OANDA 
2. Get P&L
3. Visualize some of the data